## ПЗ 5. Работа не доделанная. Сделая к 12.06.21

In [1]:
import pandas as pd
import numpy as np

In [2]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM


In [3]:
df = pd.read_csv('airline-passengers.csv')
df.info

<bound method DataFrame.info of        Month  Passengers
0    1949-01         112
1    1949-02         118
2    1949-03         132
3    1949-04         129
4    1949-05         121
..       ...         ...
139  1960-08         606
140  1960-09         508
141  1960-10         461
142  1960-11         390
143  1960-12         432

[144 rows x 2 columns]>

In [4]:
x_test[0,:]

NameError: name 'x_test' is not defined

In [ ]:
index = imdb.get_word_index()
len(index)

In [ ]:
np.max(x_train)

In [ ]:
max_features

"Мешок слов" - Bag of Word (BoW) 

http://lab314.brsu.by/kmp-lite/kmp2/JOB/CModel/BoW-Q.htm





In [ ]:
def vectorize(sequences, dimension = 10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1
  return results
 
data_train = vectorize(x_train, dimension = max_features)
data_test = vectorize(x_test, dimension = max_features)


In [ ]:

print('Построение модели...')
modelbow = Sequential()

modelbow.add(Dense(128,activation='relu',input_shape=(max_features, )))

modelbow.add(Dense(1, activation='sigmoid'))

# стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров 
modelbow.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
print('Процесс обучения...')
modelbow.fit(data_train[:10000,:], y_train[:10000],
          batch_size=batch_size,
          epochs=5, # увеличьте при необходимости
          validation_data=(data_test, y_test))

In [ ]:
modelbow.summary()

обучение на последоветельности

In [ ]:
def vectorize_seq(sequences,xLen = 100, step = 1):
  results = np.zeros((len(sequences), xLen,1))
  for i, sequence in enumerate(sequences):
    for j in range(xLen) :
       
      results[i, j,:] = sequence[j]
  return results
 
data_train_seq = vectorize_seq(x_train, xLen = 80)
data_test_seq = vectorize_seq(x_test, xLen = 80)

In [ ]:
data_test_seq.shape

In [ ]:
# https://keras.io/examples/nlp/bidirectional_lstm_imdb/

print('Построение модели...')
model = Sequential()

model.add(LSTM(128,return_sequences=False,input_shape=( 80,1)))

model.add(Dense(1, activation='sigmoid'))

# стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [ ]:
print('Процесс обучения...')
model.fit(data_train_seq, y_train,
          batch_size=100,
          epochs=3, # увеличьте при необходимости
          validation_data=(data_test_seq, y_test))

In [ ]:
# https://keras.io/examples/nlp/bidirectional_lstm_imdb/

print('Построение модели...')
model = Sequential()

model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(1, activation='sigmoid'))

# стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1, # увеличьте при необходимости
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)

LSTM  imbd c попыткой показать существенные слова (по Shaple)

https://slundberg.github.io/shap/notebooks/deep_explainer/Keras%20LSTM%20for%20IMDB%20Sentiment%20Classification.html 

## Практическое задание

<ol>
    <li>Попробуйте обучить нейронную сеть LSTM на любом другом датасете (любимый временной ряд, текст на русском (другом языке) как генератор или классификатор, или прилагаемый набор airline-passengers - пасажиропоток для авиалиний). Опишите, какой результата вы получили? Что помогло вам улучшить ее точность?

    
<li>*Попробуйте на numpy реализовать нейронную сеть архитектуры LSTM</li>
<li>*Предложите свои варианты решения проблемы исчезающего градиента в RNN</li>
</ol>